In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import xgboost as xgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

必要なライブラリをインポート後、データの読み込み

In [ ]:
import pandas as pd

train = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/train.csv.zip')
test = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/test.csv.zip')

# Idは不要なので、削除して別に変数化し、スコア提出時に使用
train_Id = train.Id
test_Id = test.Id

# Id列削除
train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)

In [ ]:
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
import seaborn as sns; sns.set(style="ticks", color_codes=True)

from datetime import datetime
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# 最大カラム数を100に拡張(デフォルトだと省略されてしまうので)
# 常に全ての列（カラム）を表示
pd.options.display.max_columns = None
pd.options.display.max_rows = 80

# 小数点2桁で表示(指数表記しないように)
pd.options.display.float_format = '{:.2f}'.format
%matplotlib inline
#ワーニングを抑止
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

テストデータ、学習データの確認

In [ ]:
print('Size of train data', train.shape)
print('Size of test data', test.shape)

In [ ]:
train.shape

In [ ]:
train.info()

データの数値項目の表示

In [ ]:
train.describe()

In [ ]:
train.describe(include='O')

学習データの記録数、ユニーク数、最頻値の出現回数

In [ ]:
train["revenue"].describe()

目的変数の確認(revenue)

In [ ]:
#目的変数であるrevenueのヒストグラムとQ-Qプロットを表示する
# 分布確認
fig = plt.figure(figsize=(10, 4))
plt.subplots_adjust(wspace=0.4)

# ヒストグラム
ax = fig.add_subplot(1, 2, 1)
sns.distplot(train['revenue'], ax=ax)

# QQプロット
ax2 = fig.add_subplot(1, 2, 2)
stats.probplot(train['revenue'], plot=ax2)

plt.show()

# 変換後の要約統計量表示
print(train['revenue'].describe())
print("------------------------------")
print("歪度: %f" % train['revenue'].skew())
print("尖度: %f" % train['revenue'].kurt())

revenueを対数変換

In [ ]:
# 学習データをコピーし、新たなdataframeで検証
df = train.copy()

#目的変数の対数log(x+1)をとる
df['revenue'] = np.log1p(df['revenue'])

# 標準化(平均0, 分散1)
scaler=StandardScaler()
df['revenue']=scaler.fit_transform(df[['revenue']])

# 分布確認
fig = plt.figure(figsize=(10, 4))
plt.subplots_adjust(wspace=0.4)

# ヒストグラム
ax = fig.add_subplot(1, 2, 1)
sns.distplot(df['revenue'], ax=ax)

# QQプロット
ax2 = fig.add_subplot(1, 2, 2)
stats.probplot(df['revenue'], plot=ax2)

plt.show()

# 変換後の要約統計量表示
print(df['revenue'].describe())
print("------------------------------")
print("歪度: %f" % df['revenue'].skew())
print("尖度: %f" % df['revenue'].kurt())

revenueを正規化（最大1、最小0）

In [ ]:
# 学習データをコピーし、新たなdataframeで検証
df = train.copy()

# Min-Max変換(正規化(最大1, 最小0))
scaler=MinMaxScaler()
df['revenue']=scaler.fit_transform(df[['revenue']])

# 分布確認
fig = plt.figure(figsize=(10, 4))
plt.subplots_adjust(wspace=0.4)

# ヒストグラム
ax = fig.add_subplot(1, 2, 1)
sns.distplot(df['revenue'], ax=ax)

# QQプロット
ax2 = fig.add_subplot(1, 2, 2)
stats.probplot(df['revenue'], plot=ax2)

plt.show()

# 変換後の要約統計量表示
print(df['revenue'].describe())
print("------------------------------")
print("歪度: %f" % df['revenue'].skew())
print("尖度: %f" % df['revenue'].kurt())

In [ ]:
#  学習データをコピーし、新たなdataframeで検証
df = train.copy()

# 標準化(平均0, 分散1)
scaler=StandardScaler()
df['revenue']=scaler.fit_transform(df[['revenue']])


# 分布確認
fig = plt.figure(figsize=(10, 4))
plt.subplots_adjust(wspace=0.4)

# ヒストグラム
ax = fig.add_subplot(1, 2, 1)
sns.distplot(df['revenue'], ax=ax)

# QQプロット
ax2 = fig.add_subplot(1, 2, 2)
stats.probplot(df['revenue'], plot=ax2)

plt.show()

# 変換後の要約統計量表示
print(df['revenue'].describe())
print("------------------------------")
print("歪度: %f" % df['revenue'].skew())
print("尖度: %f" % df['revenue'].kurt())

正規化ではなく、対数変換を使用する

open dateを数値化、年と月にわける

In [ ]:
# 学習データ
# Open Dateを日付型に変換
train['pd_date'] = pd.to_datetime(train['Open Date'], format='%m/%d/%Y')
# 年のみを抽出
train['Open_Year'] = train['pd_date'].dt.strftime('%Y')
# 月のみを抽出
train['Open_Month'] = train['pd_date'].dt.strftime('%m')
train['Open Date'] = pd.to_datetime(train["Open Date"])
# train["Day"] = train["Open Date"].apply(lambda x:x.day)

# train = train.drop('kijun', axis=1)

train = train.drop('pd_date',axis=1)
train = train.drop('Open Date',axis=1)

In [ ]:
# テストデータ
# Open Dateを日付型に変換
test['pd_date'] = pd.to_datetime(test['Open Date'], format='%m/%d/%Y')
# 年のみを抽出
test['Open_Year'] = test['pd_date'].dt.strftime('%Y')
# 月のみを抽出
test['Open_Month'] = test['pd_date'].dt.strftime('%m')
test['Open Date'] = pd.to_datetime(test["Open Date"])
# test["Day"] = test["Open Date"].apply(lambda x:x.day)

# test = test.drop('kijun', axis=1)
test = test.drop('pd_date',axis=1)
test = test.drop('Open Date',axis=1)

説明変数の確認

In [ ]:
train.dtypes.value_counts()

In [ ]:
#カテゴリ変数と数値変数に分ける
cats = list(train.select_dtypes(include=['object']).columns)
nums = list(train.select_dtypes(exclude=['object']).columns)
print(f'categorical variables:  {cats}')
print(f'numerical variables:  {nums}')

In [ ]:
train.nunique(axis=0)

In [ ]:
print(f'categorical variables:  {cats}')
print(f'numerical variables:  {nums}')

名義変数、数値変数に分ける

In [ ]:
# 名義変数
nominal_list =cats
               

# 数値変数
num_list = nums

名義変数の確認

In [ ]:
columns = len(nominal_list)/2+1

fig = plt.figure(figsize=(30, 20))
plt.subplots_adjust(hspace=0.6, wspace=0.4)

for i in range(len(nominal_list)):
    ax = fig.add_subplot(columns, 2, i+1)
    sns.countplot(x=nominal_list[i], data=train, ax=ax)
    plt.xticks(rotation=45)
plt.show()

数値変数の確認

In [ ]:
columns = len(num_list)/3+1

fig = plt.figure(figsize=(30, 40))
plt.subplots_adjust(hspace=0.6, wspace=0.4)

for i in range(len(num_list)):
    ax = fig.add_subplot(columns, 3, i+1)

    train[num_list[i]].hist(ax=ax)
    ax2 = train[num_list[i]].plot.kde(ax=ax, secondary_y=True,title=num_list[i])
    ax2.set_ylim(0)
    
plt.show()

名義変数とターゲット変数の関係を確認

In [ ]:
columns = len(nominal_list)/2+1

fig = plt.figure(figsize=(20, 10))
plt.subplots_adjust(hspace=0.6, wspace=0.4)

for i in range(len(nominal_list)):
    ax = fig.add_subplot(columns, 2, i+1)

    # 回帰の場合    
    sns.boxplot(x=nominal_list[i], y=train.revenue, data=train, ax=ax)
    plt.xticks(rotation=45)
    # 分類の場合
#     sns.barplot(x = nominal_list[i], y = train.revenue, data=train, ax=ax)
plt.show()

In [ ]:
train = train.drop('Open_Month',axis=1)
test= test.drop('Open_Month',axis=1)
nominal_list.remove('Open_Month')

数値変数とターゲット変数の関係を確認

In [ ]:
columns = len(num_list)/4+1

fig = plt.figure(figsize=(30, 35))
plt.subplots_adjust(hspace=0.6, wspace=0.4)

for i in range(len(num_list)):
    ax = fig.add_subplot(columns, 4, i+1)

    # 回帰の場合    
    sns.regplot(x=num_list[i],y='revenue',data=train, ax=ax)
    plt.xticks(rotation=45)
    # 分類の場合
#     sns.barplot(x = nominal_list[i], y = train.revenue, data=train, ax=ax)
plt.show()

新しい特徴量の作成

In [ ]:
train[['City','revenue']].groupby('City').mean().plot(kind='bar')
plt.title('Mean Revenue Generated vs City')
plt.xlabel('City')
plt.ylabel('Mean Revenue Generated')

データにバラつきがあるため、平均収入を100万単位にする

In [ ]:
# Cityごとのrevenue平均値を1000000単位とする
mean_revenue_per_city = train[['City', 'revenue']].groupby('City', as_index=False).mean()
mean_revenue_per_city.head()
mean_revenue_per_city['revenue'] = mean_revenue_per_city['revenue'].apply(lambda x: int(x/1e6)) 

mean_revenue_per_city

mean_dict = dict(zip(mean_revenue_per_city.City, mean_revenue_per_city.revenue))
mean_dict

テストデータと学習データの都市の違いの確認

In [ ]:
print(train['City'].sort_values().unique())

In [ ]:
test['City'].sort_values().unique()

In [ ]:
# Cityについて、学習データとテストデータにて重複削除し、リスト化
city_train_list = list(train['City'].unique())
city_test_list = list(test['City'].unique())

共通都市を抽出

In [ ]:
l1_l2_and = set(city_train_list) & set(city_test_list)
print(l1_l2_and)
print(len(l1_l2_and))

どちらかにしかないCityを抽出

In [ ]:

l1_l2_sym_diff = set(city_test_list) ^ set(city_train_list)
print(l1_l2_sym_diff)
print(len(l1_l2_sym_diff))


テストデータのみ存在するCityの件数

In [ ]:
len(set(city_test_list).difference(city_train_list))

学習データのみ存在するCityの件数

In [ ]:
# 学習データのみ存在するCityの件数
len(set(city_train_list).difference(city_test_list))

クラスタリングにより存在しない都市の補完

各都市とp変数の関係を調べる

In [ ]:
# P変数の1つのクラスは地理的属性であると指定されているため
# 各都市のP変数の平均をプロットすると、どのP変数が都市と関連性が高いかが分かる
distinct_cities = train.loc[:, "City"].unique()

# P変数のcityごとの平均値を取得
means = []
for i in range(len(num_list)):
    temp = []
    for city in distinct_cities:
        temp.append(train.loc[train.City == city, num_list[i]].mean())  
    means.append(temp)
    
city_pvars = pd.DataFrame(columns=["city_var", "means"])
for i in range(37):
    for j in range(len(distinct_cities)):
        city_pvars.loc[i+37*j] = ["P"+str(i+1), means[i][j]]

print(city_pvars)            
# 箱ひげ図を表示
plt.rcParams['figure.figsize'] = (18.0, 6.0)
sns.boxplot(x="city_var", y="means", data=city_pvars)

学習データとテストデータの集約（まとめて操作するため）

In [ ]:
from sklearn import cluster

def adjust_cities(full_full_data, train, k):
    
    # As found by box plot of each city's mean over each p-var
    relevant_pvars =  ["P1", "P2", "P11", "P19", "P20", "P23","P30"]
    train = train.loc[:, relevant_pvars]
    
    # Optimal k is 20 as found by DB-Index plot    
    kmeans = cluster.KMeans(n_clusters=k)
    kmeans.fit(train)
    
    # Get the cluster centers and classify city of each full_data instance to one of the centers
    full_data['City_Cluster'] = kmeans.predict(full_data.loc[:, relevant_pvars])
    
    return full_data

データを再分割

In [ ]:
num_train = train.shape[0]
num_test = test.shape[0]
print(num_train, num_test)

full_data = pd.concat([train, test], ignore_index=True) 

In [ ]:
# 学習データを使用しクラスタリングを行い、その学習結果を全データに適用させる
full_data = adjust_cities(full_data, train, 20)
full_data

# City項目は不要なので削除
full_data = full_data.drop(['City'], axis=1)

In [ ]:
# Split into train and test datasets
train = full_data[:num_train]
test = full_data[num_train:]
# check the shapes 
print("Train :",train.shape)
print("Test:",test.shape)
test

In [ ]:
# Split into train and test datasets
train = full_data[:num_train]
test = full_data[num_train:]
# check the shapes 
print("Train :",train.shape)
print("Test:",test.shape)
test

In [ ]:
train[['City_Cluster','revenue']].groupby('City_Cluster').mean().plot(kind='bar')
plt.title('Mean Revenue Generated vs City Cluster')
plt.xlabel('City Cluster')
plt.ylabel('Mean Revenue Generated')

In [ ]:
mean_revenue_per_city = train[['City_Cluster', 'revenue']].groupby('City_Cluster', as_index=False).mean()
mean_revenue_per_city.head()
mean_revenue_per_city['revenue'] = mean_revenue_per_city['revenue'].apply(lambda x: int(x/1e6)) 

mean_revenue_per_city

mean_dict = dict(zip(mean_revenue_per_city.City_Cluster, mean_revenue_per_city.revenue))
mean_dict

In [ ]:
city_rev = []

for i in full_data['City_Cluster']:
    for key, value in mean_dict.items():
        if i == key:
            city_rev.append(value)
            
df_city_rev = pd.DataFrame({'city_rev':city_rev})
full_data = pd.concat([full_data,df_city_rev],axis=1)
full_data.head

# 値の追加
nominal_list.extend(['City_Cluster'])
# 値の削除
nominal_list.remove('City')

カテゴリー変数のエンコード
カテゴリが二つしかない変数にはエンコード、三つ以上のものにはワンホットを採用

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_count = 0

# Iterate through the columns
# for col in application_full_data:
for i in range(len(nominal_list)):    
    
#     if application_full_data[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(full_data[nominal_list[i]].unique())) <= 2:
            # full_data on the full_dataing data
            le.fit(full_data[nominal_list[i]])
            # Transform both full_dataing and testing data
            full_data[nominal_list[i]] = le.transform(full_data[nominal_list[i]])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

In [ ]:
# one-hot encoding of categorical variables
full_data = pd.get_dummies(full_data)
print('full_dataing Features shape: ', full_data.shape)

In [ ]:
def tukey_outliers(x):
    q1 = np.percentile(x,25)
    q3 = np.percentile(x,75)
    
    iqr = q3-q1
    
    min_range = q1 - iqr*1.5
    max_range = q3 + iqr*1.5
    
    outliers = x[(x<min_range) | (x>max_range)]
    return outliers

In [ ]:
columns = len(num_list)/4+1

# boxplot
fig = plt.figure(figsize=(15,20))
plt.subplots_adjust(hspace=0.2, wspace=0.8)
for i in range(len(num_list)):
    ax = fig.add_subplot(columns, 4, i+1)
    sns.boxplot(y=full_data[num_list[i]], data=full_data, ax=ax)
plt.show()

In [ ]:
skewed_data = train[num_list].apply(lambda x: skew(x)).sort_values(ascending=False)
skewed_data[:10]

In [ ]:
# Split into train and test datasets
train = full_data[:num_train]
test = full_data[num_train:]
# check the shapes 
print("Train :",train.shape)
print("Test:",test.shape)

revenueと相関の高い変数のトップ10を確認

In [ ]:
# Find correlations with the target and sort
correlations = train.corr()['revenue'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

In [ ]:
# 相関が高い10項目のみ抽出
correlations = train.corr()
# 絶対値で取得
correlations = abs(correlations)

cols = correlations.nlargest(10,'revenue')['revenue'].index
cols

In [ ]:
# 相関が高い10項目のみ抽出
train = train[cols]

#学習データを目的変数とそれ以外に分ける
train_X = train.drop("revenue",axis=1)
train_y = train["revenue"]

#revenueを対数変換する 
train_y = np.log1p(train_y)

#テストデータを学習データのカラムのみにする 
tmp_cols = train_X.columns
test_X = test[tmp_cols]

#それぞれのデータのサイズを確認
print("train_X: "+str(train_X.shape))
print("train_y: "+str(train_y.shape))
print("test_X: "+str(test_X.shape))

In [ ]:
#訓練データとモデル評価用データに分けるライブラリ
from sklearn.model_selection import train_test_split

#フォールドアウト法により、学習データとテストデータに分割 
(train_x, valid_x, train_y, valid_y) = train_test_split(train_X, train_y , test_size = 0.3 , random_state = 0)

GDBTによる識別

In [ ]:
# #特徴量と目的変数をxgboostのデータ構造に変換する
# dtrain = xgb.DMatrix(train_x, label=train_y)
# dvalid = xgb.DMatrix(valid_x,label=valid_y)

# #パラメータを指定してGBDT
# num_round = 5000
# evallist = [(dvalid, 'eval'), (dtrain, 'train')]

# evals_result = {}

# #パラメータ
# param = {
#             'max_depth': 3,
#             'eta': 0.01,
#             'objective': 'reg:squarederror',
# }

# #学習の実行
# bst = xgb.train(
#                         param, dtrain,
#                         num_round,
#                         evallist,
#                         evals_result=evals_result,
#                         # 一定ラウンド回しても改善が見込めない場合は学習を打ち切る
#                         early_stopping_rounds=1000)

lightGBM

In [ ]:
# lightGBMによる予測
lgb_train = lgb.Dataset(train_x, train_y)
lgb_eval = lgb.Dataset(valid_x, valid_y, reference=lgb_train)

# LightGBM parameters
params = {
        'task' : 'train',
        'boosting_type' : 'gbdt',
        'objective' : 'regression',
        'metric' : {'l2'},
        'num_leaves' : 31,
        'learning_rate' : 0.1,
        'feature_fraction' : 0.9,
        'bagging_fraction' : 0.8,
        'bagging_freq': 5,
        'verbose' : 0,
        'n_jobs': 2
}

gbm = lgb.train(params,
            lgb_train,
            num_boost_round=100,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)

prediction_lgb = np.exp(gbm.predict(test_X))

In [ ]:
# prediction_log = xgb.DMatrix(test_X)
# # 対数を戻す
# prediction =np.exp(bst.predict(prediction_log)) 


# 予測した値を提出用CSVファイル(submissionファイル)に書き出し
submission = pd.DataFrame({"Id":test_Id, "Prediction":prediction_lgb})
submission.to_csv("submission.csv", index=False)